# New Version of Comment Labeling Using Keywords

In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
path = "C:/Users/doosti/Dropbox (Chapman)/Research/Research Projects/Fitness/Data/"
file = "comments_negative_010424.csv"

In [5]:
comments_labeled = pd.read_csv(os.path.join(path, file))
comments = pd.read_csv(os.path.join(path, "merged_comments.csv"))
videos = pd.read_csv(os.path.join(path, "videos_101223.csv"))
videos_trends = pd.read_csv(os.path.join(path, "final_yoga_videos_tends_101023.csv"))
trends = pd.read_csv(os.path.join(path, "final_yoga_channels_trends_091523.csv"))

In [6]:
# merging comments and labels
comments.index = np.arange(len(comments))
comments_labeled = comments_labeled[:-2999].copy() # removing the extra labeled comments
comments_labeled.index = comments_labeled['doc_id']

comments.loc[comments_labeled.index, 'negative'] = comments_labeled['negative']
print(comments['negative'].value_counts())

negative
0.0    826955
1.0       525
Name: count, dtype: int64


In [7]:
# threshold = 11
comments = comments[comments.video_id.str.len() == 11].copy()
print(comments.shape)
# drop rows with missing year, month, or day
comments_labeled = comments.dropna(subset=['comment_year', 'comment_month', 'comment_day']).copy()
print(comments_labeled.shape)
print(comments_labeled.isnull().sum())

(827566, 9)
(827552, 9)
video_id           0
user_id            0
comment_text     100
no_likes           0
no_replies         0
comment_year       0
comment_month      0
comment_day        0
negative         100
dtype: int64


In [8]:
# Create a function to format and convert the date
def to_datetime_with_padding(year, month, day):
    # Pad month and day with zeros if necessary
    month = f"{month:02d}"
    day = f"{day:02d}"
    # Combine and convert to datetime
    date_str = f"{year}-{month}-{day}"
    return pd.to_datetime(date_str)

comments_labeled['comment_year'] = comments_labeled.comment_year.astype(int)
comments_labeled['comment_month'] = comments_labeled.comment_month.astype(int)
comments_labeled['comment_day'] = comments_labeled.comment_day.astype(int)

# Apply the function to each row
comments_labeled['date'] = comments_labeled.apply(lambda row: to_datetime_with_padding(row["comment_year"], row["comment_month"], row["comment_day"]), axis=1)

In [34]:
#comments_labeled.fillna(0).to_csv(os.path.join(path, "negative_comments_010424.csv"), index=False)

In [9]:
# merging with video dataset
#comments_labeled = 
comments_labeled['creator_id'] = comments_labeled.merge(videos,how='left',on='video_id').creator_id.values
comments_labeled = comments_labeled.dropna(subset=['creator_id'])
print(comments_labeled.isnull().sum())

video_id           0
user_id            0
comment_text     100
no_likes           0
no_replies         0
comment_year       0
comment_month      0
comment_day        0
negative         100
date               0
creator_id         0
dtype: int64


In [11]:
channel_day = comments_labeled.fillna(0).groupby(['creator_id','date']).negative.value_counts().unstack().reset_index()
channel_day = channel_day.fillna(0)
channel_day = channel_day.rename(columns={0:'none', 1:'negative'})
print(channel_day.shape)
print(channel_day.creator_id.nunique())
print(channel_day.head())


(48673, 4)
71
negative           creator_id       date  none  negative
0         262h9yTG1f03Nobo501 2020-11-05   1.0       0.0
1         262h9yTG1f03Nobo501 2020-11-11  28.0       0.0
2         262h9yTG1f03Nobo501 2020-11-12  46.0       0.0
3         262h9yTG1f03Nobo501 2020-11-13  42.0       0.0
4         262h9yTG1f03Nobo501 2020-11-14  46.0       0.0


In [12]:
channel_day.to_csv(os.path.join(path, "channel_day_negative_010424.csv"), index=False)